<a href="https://colab.research.google.com/github/alexelax/Smart-Traffic-Lights/blob/Colab-notebooks/YOLOv5_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: VARIABILI

In [ ]:
import os
# se true scarica i punti dal progetto SmartTrafficLight su git se no usa i pt pretrainati con coco
#@markdown Usare il vecchio pt?
USE_OLD_PT = "False"   #@param ["False", "True"] 
OLD_PT_PATH="https://github.com/alexelax/Smart-Traffic-Lights/tree/YOLO/YOLOv5/_infer/best_n.pt" #@param {type: 'string'}

#@markdown modello da usare
PT_NAME= "yolov5n.pt"#@param ['yolov5n.pt', 'yolov5s.pt', 'yolov5m.pt', 'yolov5l.pt','yolov5x.pt'] {allow-input: false}



#li setto anche nelle variabili d'ambiente che non si sa mai...
os.environ['USE_OLD_PT']=USE_OLD_PT    
os.environ['OLD_PT_PATH']=OLD_PT_PATH    
os.environ['PT_NAME']=PT_NAME



# Step 1: Install Requirements

In [ ]:
%%capture 
#DIO GRAZIE DEL CAPUTRE!! ( nasconde l'output! ) 

!cd /
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

#import torch
#import os
#from IPython.display import Image, clear_output  # to display images
#print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

#scarico i vecchi PT ( trainati )  se richiesto
import os
import urllib.request
print(os.environ)
if os.environ['USE_OLD_PT']=="True":
  urllib.request.urlretrieve(os.environ['OLD_PT_PATH'],"/content/yolov5/"+os.environ['PT_NAME'])

# Step 2: Download Dataset

In [ ]:
%%capture
WORKSPACE_ID="traffic-light-sgslp"
PROJECT_ID="traffic-detection-0dhod"
api_key="nhwCFkPkiM0Ges2WAtNV"

#WORKSPACE_ID="hands-fnbo9"
#PROJECT_ID="mani-seryz"




from roboflow import Roboflow
import os
os.environ["DATASET_DIRECTORY"] = "/content/datasets"
rf = Roboflow(api_key=api_key)
project = rf.workspace(WORKSPACE_ID).project(PROJECT_ID)

#prende l'ultima versione disponibile su roboflow 
#project.versions()[0].version è una stringa con alla fine l'id 
VERSION =int(project.versions()[0].version.split("/")[-1])   
dataset = project.version(VERSION).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/Traffic-Detection-1 in yolov5pytorch:: 100%|██████████| 212/212 [00:00<00:00, 1930.00it/s]


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location` -> variabile di sistema che punta già alla cartella scaricata da roboflow
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
%rm -rf /content/yolov5/runs/
import os
os.environ['dataset_location']=dataset.location
#NON SO XKE, prima funzionava anche senza di questo...
#la variabile di python dataset.location è settata sulla cartella giusta ma usando "{dataset.location}" nella riga sotto non veniva trovata
#allora la metto in una variabile d'ambiente e la valuto ( con il $ ) in fase di richiamo del train ( come con il PT_NAME )

!python train.py --img 640 --batch 4 --epochs 3000 --data ${dataset_location}/data.yaml --weights ${PT_NAME} --cache

#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

train: weights=yolov5n.pt, cfg=, data=/content/datasets/Traffic-Detection-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3000, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-102-ge4d8360 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>